# particle_interp_compare

compare particle simulations in different fieldsets with a single particle

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
from datetime import timedelta

import cartopy
import cartopy.crs as ccrs
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from parcels import FieldSet, ParticleSet, JITParticle, AdvectionRK4, ErrorCode, plotTrajectoriesFile, ParticleFile

from utils import create_path, add_noise
from parcels_utils import get_file_info
from plot_utils import plot_trajectories

In [ ]:
dir1km = "current_netcdfs/west_coast_1km_hourly/"

files = [
    get_file_info(dir1km + "tijuana_river_lin.nc", 1, name="lin"),
    get_file_info(dir1km + "tijuana_river_lin_aggr.nc", 1, name="lin_aggr")
]

st = files[0]["timerng"][0]
ed = files[0]["timerng"][1]
for f in files:
    assert f["timerng"][0] == st
    assert f["timerng"][1] == ed

In [ ]:
do_sim = True
end_time = files[0]["timerng"][1]
# this was the very specific spawn time for particles that collided with land
spawn_time = np.datetime64("2020-06-17T09:00:00")
max_var = 0.003
spawn_loc_base = np.array([32.551707, -117.138]) # (lat, lon)
spawn_locs = add_noise(spawn_loc_base, max_var, repeat=20)
simulation_dt = timedelta(minutes=5)
out_dt = timedelta(hours=1)
save_dir = create_path("particledata/testcompare")

def DeleteParticle(particle, fieldset, time):
    particle.delete()

now i could just stuff everything into a single ParticleSet for every different spawn location

but then it becomes annoying to filter out ones that start at a specific location

In [ ]:
for f in files:
    f["psets"] = np.empty(len(spawn_locs), dtype=ParticleSet)
    f["pfiles"] = np.empty(len(spawn_locs), dtype=ParticleFile)
    
    for i in range(len(spawn_locs)):
        f["psets"][i] = ParticleSet(fieldset=f["fs"], pclass=JITParticle, lon=[spawn_locs[i][1]], lat=[spawn_locs[i][0]], time=[spawn_time])
        save_path = f"{save_dir}/{f['name']}{i}.nc"
        f["pfiles"][i] = f["psets"][i].ParticleFile(save_path, outputdt=out_dt)

In [ ]:
if do_sim:
    for f in files:
        for i in range(len(spawn_locs)):
            f["psets"][i].execute(
                        AdvectionRK4,
                        endtime=end_time,
                        dt=simulation_dt,
                        recovery={ErrorCode.ErrorOutOfBounds: DeleteParticle},
                        output_file=f["pfiles"][i]
                    )
            f["pfiles"][i].export()
            f["pfiles"][i].close()

In [ ]:
print(files[0]["domain"])

In [ ]:
mod_domain = {
    'S': 32.545,
    'N': 32.585,
    'W': -117.158,
    'E': -117.13
}
# mod_domain = {
#     'S': 32.5495,
#     'N': 32.555,
#     'W': -117.15,
#     'E': -117.135
# }
# mod_domain = {
#     'S': 32.575,
#     'N': 32.5766,
#     'W': -117.145,
#     'E': -117.140
# }

for i in range(len(spawn_locs)):
    ps = np.array([f["pfiles"][i].name for f in files])
    print(f"starting at (lat, lon) {spawn_locs[i]}")
#     plot_trajectories(ps, mod_domain)

In [ ]:
for f in files:
    ps = np.array([p.name for p in f["pfiles"]])
    print(f"paths for {f['name']}")
    plot_trajectories(ps, mod_domain, legend=False, scatter=False)

wack ass coordinates

[  32.55096753 -117.13918667]

[  32.55072813 -117.13842626]